### Импорт необходимых библиотек

In [42]:
import os
import sqlite3
import time

import pandas as pd
import requests
from tqdm import tqdm

In [2]:
limit_ = 6000  # 6500 уже даёт статус 504
offset_ = 1
url = f'https://наш.дом.рф/сервисы/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

### GET запрос для получения данных

In [3]:
res = requests.get(url)
objects_data = res.json()

In [4]:
len(objects_data['data']['list'])

6000

### Получение списка id объектов для получения данных для каждого из объектов

In [5]:
objects_ids = [str(object_data['objId']) for object_data in objects_data['data']['list']]

In [6]:
len(objects_ids)

6000

### Получение списка расширенных данных по каждому из объектов

In [9]:
def get_every_object_data(objects_ids: list) -> list:
    result_list = []
    for object_id in tqdm(objects_ids):
        try:
            url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{object_id}'
            resp = requests.get(url)
            data = resp.json()['data']
            if type(data) == dict:  # для отсеивания "Не получено данных"
                result_list.append(data)
            time.sleep(.5)  # чтобы не забанили IP-адрес
        except Exception:  # т.к. обращение идёт к стороннему сервису, что может вызывать ошибки
            continue
    return result_list

In [14]:
single_objects_data_list = get_every_object_data(objects_ids)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6000/6000 [1:13:02<00:00,  1.37it/s]


### Создание pandas.DataFrame

In [15]:
df = pd.json_normalize(single_objects_data_list)
df.head()

,id,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,...,metro.time,metro.isWalk,metro.colors,greenAreaIndexValue,airQualityIndexValue,complexShortNm,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13391,28101,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,Другое,По желанию заказчика,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13397,28141,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13398,28141,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13399,28141,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23682,64486,47,д Агалатово,10,10,54,2022-06-30,Блоки,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Сохранение pandas.DataFrame в excel

In [16]:
df.to_excel('df_excel.xlsx', index=False)

### Сохранение pandas.DataFrame в pickle

In [17]:
df.to_pickle('df_pickle.pkl')

### Сохранение pandas.DataFrame в базу данных

In [26]:
conn = sqlite3.connect('df_to_database')
df.to_sql('objects', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

In [ ]:
single_object_data

### Скрипт для сохранения фото объектов

In [32]:
single_objects_data_list[0]['photoRenderDTO']  # список описания фотографий объекта

[{'objId': 13391,
  'pdId': 28101,
  'rpdId': 592252,
  'devId': 306,
  'ordNum': 1,
  'objRenderPhotoSize': 310482,
  'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/1eca0ffa-fb5f-4b30-a239-632c4c79ecd2',
  'objRenderPhotoDttm': '19-08-2020 15:59',
  'objRenderPhotoHiddenFlg': 0,
  'objRenderPhotoNm': 'IMG_6313_2.JPG',
  'objReadyDesc': 'Строится',
  'objBuildTypeShortDesc': 'Жилое',
  'knFlg': 1,
  'loadDttm': '15-05-2022 00:56'},
 {'objId': 13391,
  'pdId': 28101,
  'rpdId': 592252,
  'devId': 306,
  'ordNum': 2,
  'objRenderPhotoSize': 1014913,
  'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/a7d88189-046a-49bb-834a-d1745b751eec',
  'objRenderPhotoDttm': '12-05-2021 16:55',
  'objRenderPhotoHiddenFlg': 0,
  'objRenderPhotoNm': 'IMG_20210512_094028.jpg',
  'objReadyDesc': 'Строится',
  'objBuildTypeShortDesc': 'Жилое',
  'knFlg': 1,
  'loadDttm': '15-05-2022 00:56'},
 {'objId': 13391,
  'pdId': 28101,
  'rpdId': 592252,
  'devId': 3

### Получение и сохранение фото объектов

In [61]:
def get_object_images(objects_data: list) -> None:
    for obj in tqdm(objects_data): 
        object_img_list = obj['photoRenderDTO']
        object_id = str(object_img_list[0]['objId'])
        object_address = obj['address']
        os.mkdir(object_id)
        for img in object_img_list:
            img_url = img['objRenderPhotoUrl']
            img_name = img['objRenderPhotoNm']
            try:
                img_resp = requests.get(img_url)
                with open('/'.join((object_id, img_name)), 'wb') as f:
                    f.write(img_resp.content)
                with open('/'.join((object_id, 'address.txt')), 'w') as f:
                    f.write(object_address)            
                del img_resp
            except Exception:
                continue

In [62]:
get_object_images(single_objects_data_list[:10])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.29s/it]
